# Main 1 : Covid 19

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.read.csv("covid19.csv", header=True, inferSchema=True)
import pandas as pd
import numpy as np

dff = df.toPandas()
#The dataSet is too big .. We will only work with 500 rows !! just to test the streaming 
#print(dff.head(500).to_csv("covid19_10.csv",index=False))
df1 = spark.read.csv("covid19_10.csv", header=True, inferSchema=True)


In [4]:
df.columns

['SNo',
 'ObservationDate',
 'Province',
 'Country',
 'Last_Update',
 'Confirmed',
 'Deaths',
 'Recovered',
 'Active_case']

In [5]:
df.show(2)

+---+---------------+--------+-------+---------------+---------+------+---------+-----------+
|SNo|ObservationDate|Province|Country|    Last_Update|Confirmed|Deaths|Recovered|Active_case|
+---+---------------+--------+-------+---------------+---------+------+---------+-----------+
|  1|     01/22/2020|   Anhui|  China|1/22/2020 17:00|        1|     0|        0|          1|
|  2|     01/22/2020| Beijing|  China|1/22/2020 17:00|       14|     0|        0|         14|
+---+---------------+--------+-------+---------------+---------+------+---------+-----------+
only showing top 2 rows



Step maps a unit of time in the real world. In this case 1 step is 1 hour of time. So we can assume for this example that we have another job that runs every hour and gets all the transactions in that time frame.

In [6]:
df.groupBy("Last_Update").count().show(3)

+-------------------+-----+
|        Last_Update|count|
+-------------------+-----+
|2020-02-10T13:23:06|    1|
|2020-02-10T19:03:02|    3|
|2020-02-21T13:03:09|    1|
+-------------------+-----+
only showing top 3 rows



We can therefore save the output of that job by filtering on each step and saving it to a separate file. 

In [7]:
#%%time
 #Countries = df1.select("Active_case").distinct().collect()
 #for Active_case in Countries[:]:
  #  _df = df1.where(f"Active_case = {Active_case[0]}")
   # #by adding coalesce(1) we save the dataframe to one file
    #_df.coalesce(1).write.mode("append").option("header", "true").csv("data")
    

In [8]:
!cd data/ && dir

 Le volume dans le lecteur C n'a pas de nom.
 Le num‚ro de s‚rie du volume est 9E1C-407B

 R‚pertoire de c:\sparkProjectFolder\SUAS-Project-Spark-Streaming\data

20/01/2021  19:29    <DIR>          .
20/01/2021  19:29    <DIR>          ..
20/01/2021  19:29                12 .part-00000-01d0e021-4bab-4001-8387-8815fddab4a9-c000.csv.crc
20/01/2021  19:28                12 .part-00000-03de0800-a232-4982-917f-9209102edc11-c000.csv.crc
20/01/2021  19:28                12 .part-00000-0582b78a-bb40-4b4d-822b-16953efd67cf-c000.csv.crc
20/01/2021  19:29                12 .part-00000-06429bfa-aa4a-470e-b14b-0555a0142e15-c000.csv.crc
20/01/2021  19:28                12 .part-00000-0675b652-d945-4809-a15f-21d1e1ccff4b-c000.csv.crc
20/01/2021  19:28                12 .part-00000-071deb1a-841f-4144-8e62-aadffd38c98b-c000.csv.crc
20/01/2021  19:29                12 .part-00000-0aa3c307-579c-41d1-bd2b-4ca5bf9defaf-c000.csv.crc
20/01/2021  19:29                12 .part-00000-0d22b1a7-5419-46ea-8ff0-59b

20/01/2021  19:28               150 part-00000-95d05e2f-dce7-489f-9aa0-34b83fc723fc-c000.csv
20/01/2021  19:28               204 part-00000-95dc865d-c1fb-4659-8bef-15f20df8224c-c000.csv
20/01/2021  19:28               145 part-00000-96ac8b70-bd6b-4619-b9d7-fcbbb81b9f65-c000.csv
20/01/2021  19:28             1ÿ201 part-00000-98926916-164e-4e55-8175-7ddb429e2475-c000.csv
20/01/2021  19:29               149 part-00000-9e728c31-8eec-4009-9352-b8b6e25eeedf-c000.csv
20/01/2021  19:28               149 part-00000-9f1817f9-6cf9-454a-85bc-493e03994c39-c000.csv
20/01/2021  19:29               153 part-00000-a1697636-15df-46de-955d-3dd19e3839eb-c000.csv
20/01/2021  19:29               149 part-00000-a225f480-632f-4ee9-b354-71274b71eb24-c000.csv
20/01/2021  19:28               152 part-00000-a354590e-54e6-4e53-bce8-294522d0c572-c000.csv
20/01/2021  19:28               201 part-00000-a4ae9f09-e670-4d7a-853b-640231281892-c000.csv
20/01/2021  19:28               147 part-00000-a66eea55-af58-465a-b8f4

In [9]:
part = spark.read.csv(
    "data/part-00000-01d0e021-4bab-4001-8387-8815fddab4a9-c000.csv",
    header=True,
    inferSchema=True,
)

In [10]:
part.groupBy("Country").count().show()

+-------+-----+
|Country|count|
+-------+-----+
|  China|    2|
+-------+-----+



Let’s create a streaming version of this input, we'll read each file one by one as if it was a stream.

In [11]:
dataSchema = df1.schema

In [12]:
dataSchema

StructType(List(StructField(SNo,IntegerType,true),StructField(ObservationDate,StringType,true),StructField(Province,StringType,true),StructField(Country,StringType,true),StructField(Last_Update,StringType,true),StructField(Confirmed,IntegerType,true),StructField(Deaths,IntegerType,true),StructField(Recovered,IntegerType,true),StructField(Active_case,DoubleType,true)))

*maxFilesPerTrigger* allows you to control how quickly Spark will read all of the files in the folder. 
In this example we're limiting the flow of the stream to one file per trigger.

In [13]:
streaming = (
    spark.readStream.schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .csv("data/")
)

Let's set up a transformation.

The Confirmed column is the recipient ID of the transaction.

In [14]:
confirmed_Country = streaming.groupBy("Confirmed").count().orderBy(F.desc("Confirmed"))

Now that we have our transformation, we need to specify an output sink for the results. For this example, we're going to write to a memory sink which keeps the results in memory.

We also need to define how Spark will output that data. In this example, we'll use the complete output mode (rewriting all of the keys along with their counts after every trigger).

In this example we won't include activityQuery.awaitTermination() because it is required only to prevent the driver process from terminating when the stream is active.

So in order to be able to run this locally in a notebook we won't include it.

In [15]:
activityQuery = (
    confirmed_Country.writeStream.queryName("confirmed_Country")
    .format("memory")
    .outputMode("complete")
    .start()
)

# include this in production
#activityQuery.awaitTermination()

import time

for x in range(10):
    _df = spark.sql(
        "SELECT * FROM confirmed_Country"
    )
    if _df.count() > 0:
        _df.show(10)
    time.sleep(0.5)

+---------+-----+
|Confirmed|count|
+---------+-----+
|      151|    1|
|     null|    1|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|      151|    1|
|     null|    1|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|      151|    1|
|     null|    1|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|      173|    1|
|      151|    1|
|     null|    2|
+---------+-----+



Check if stream is active

In [ ]:
spark.streams.active[0].isActive

In [ ]:
activityQuery.status

If we  want to turn off the stream we'll run activityQuery.stop() to reset the query for testing purposes.

In [98]:
activityQuery.stop()